Links:
- https://cornell.app.box.com/s/tblu711mnoajszul8x4kmpjxoy30ajvb
- `/nfs/chess/user/chess_id3a/CHAP_tomo/`
- https://github.com/CHESSComputing/ChessAnalysisPipeline

Copy 

In [4]:
from jinja2 import Template
import yaml,glob,os

def Load(filename):
    with open(filename, 'r') as f: return f.read()

def Save(filename,body):
    os.makedirs(os.path.dirname(filename),exist_ok=True)
    with open(filename, 'w') as f: return f.write(body)

# load "arguments" (user should be define) from file
args=yaml.safe_load(Load("args.yaml"))['args']

# load spec file
spec_file=args["spec_file"]
spec=[line for line in Load(spec_file).splitlines() if line]

# find the dark,brigh and tomo from *.par file
par_filenames=list(glob.glob(os.path.dirname(spec_file) + "/*.par"))
assert(len(par_filenames)==1)
par=[[token for token in line.split()] for line in Load(par_filenames[0]).splitlines() if line]
dark,bright,tomo=zip(*[(I-2,I-1,I) for I,it in enumerate(par) if it[-1]=="tomo"])
assert(len(tomo)==1)

SCAN_N=3 # i could get this number from the *.json file
dark,bright,tomo=par[dark[0]][SCAN_N],par[bright[0]][SCAN_N],par[tomo[0]][SCAN_N]

args["tomo_scan_number"]=tomo
args["darkfield_scan_number"]=dark
args["brightfield_scan_number"]=bright

# complete the template
for template_filename in glob.glob("templates/*.yaml"):
    template=Template(Load(template_filename))
    body=template.render(args)
    output_filename=os.path.join(args['output_folder'],os.path.basename(template_filename))
    Save(output_filename,body)
    print(f"saved {output_filename}")

saved ./out/map.yaml
saved ./out/pipeline.yaml
saved ./out/tomo_detector_retiga.yaml
